In [2]:
#データのロード
import json
import MeCab 
with open('clean_list.json', 'r') as f:
    clean_list = json.load(f)


In [3]:
#前処理
corpus = [value["gist"] for value in clean_list.values()]

In [5]:
clean_corpus = [MeCab.Tagger("-Owakati").parse(text).split() for text in corpus]

In [8]:
from collections import defaultdict
frequency = defaultdict(int)
for text in clean_corpus:
    for token in text:
        frequency[token] += 1

processed_corpus = [[token for token in text if frequency[token] > 1] for text in clean_corpus]

In [9]:
print(processed_corpus[0])

['離婚', '請求', 'に', '附帯', 'し', 'て', '財産', '分与', 'の', '申立て', 'が', 'さ', 'れ', 'た', '場合', 'に', 'おい', 'て', '当事', '者', 'が', '婚姻', '中', 'に', 'その', '双方', 'の', '協力', 'に', 'よっ', 'て', '得', 'た', 'もの', 'と', 'し', 'て', '分与', 'を', '求める', '財産', 'の', '一部', 'に', 'つき', '財産', '分与', 'に', 'つい', 'て', 'の', '裁判', 'を', 'し', 'ない', 'こと', 'は', '許さ', 'れ', 'ない']


In [12]:
from gensim import corpora

In [13]:
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary.token2id)

{'おい': 0, 'が': 1, 'こと': 2, 'さ': 3, 'し': 4, 'その': 5, 'た': 6, 'つい': 7, 'つき': 8, 'て': 9, 'と': 10, 'ない': 11, 'に': 12, 'の': 13, 'は': 14, 'もの': 15, 'よっ': 16, 'れ': 17, 'を': 18, '一部': 19, '中': 20, '分与': 21, '協力': 22, '双方': 23, '場合': 24, '婚姻': 25, '当事': 26, '得': 27, '求める': 28, '申立て': 29, '者': 30, '裁判': 31, '許さ': 32, '請求': 33, '財産': 34, '附帯': 35, '離婚': 36, '、': 37, 'する': 38, 'よる': 39, '保険': 40, '健康': 41, '処分': 42, '前': 43, '号': 44, '対し': 45, '平成': 46, '年': 47, '所定': 48, '扶養': 49, '改正': 50, '旨': 51, '条': 52, '法': 53, '法律': 54, '第': 55, '等': 56, '組合': 57, '行う': 58, '被': 59, '親族': 60, '該当': 61, '資格': 62, '通知': 63, '関する': 64, '項': 65, '（': 66, '）': 67, '１': 68, '１８９': 69, '２６': 70, '６９': 71, 'あっ': 72, 'い': 73, 'だっ': 74, 'とも': 75, 'なかっ': 76, 'より': 77, 'れる': 78, '争わ': 79, '事件': 80, '事案': 81, '交通': 82, '人': 83, '傷害': 84, '及び': 85, '員': 86, '完全': 87, '広汎': 88, '影響': 89, '心神': 90, '性': 91, '未遂': 92, '殺人': 93, '殺意': 94, '気付い': 95, '状態': 96, '発達': 97, '精神': 98, '耗弱': 99, '能力': 100, '被告': 101, '被害': 102, '認

In [15]:
text = "防犯防犯に類似した事件"
bow = dictionary.doc2bow(MeCab.Tagger("-Owakati").parse(text).split())
print(bow)
# , '防犯': 760, '類似': 761, '余儀

[(4, 1), (6, 1), (12, 1), (80, 1), (760, 2), (761, 1)]


In [16]:
dictionary.save('dictionary.dict')

In [35]:
#word2vecで学習
from gensim.models import word2vec
model = word2vec.Word2Vec(processed_corpus, vector_size=1000, min_count=3, window=5, epochs=50)

In [36]:
model.save('word2vec_1000.model')

In [33]:
model = word2vec.Word2Vec.load('word2vec.model')
print(model.wv.most_similar(positive=["殺人"]))

[('強盗', 0.8354291319847107), ('強姦', 0.7709787487983704), ('致傷', 0.7623990178108215), ('傷人', 0.7319909334182739), ('未遂', 0.715047299861908), ('放火', 0.6676826477050781), ('遺棄', 0.6603472828865051), ('射殺', 0.6567540764808655), ('窃盗', 0.6426176428794861), ('殺害', 0.6274630427360535)]


In [49]:
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(processed_corpus)]
model = doc2vec.Doc2Vec(documents, vector_size=500, window=5, min_count=3, epochs=50)



In [50]:
model.save('doc2vec_500.model')

In [55]:
model = doc2vec.Doc2Vec.load('doc2vec_500.model')
# Example usage of the model
similar_words = model.infer_vector(MeCab.Tagger("-Owakati").parse("学校学校での事件").split())
print(model.docvecs.most_similar([similar_words]))


[(17153, 0.4842104911804199), (4025, 0.4675421416759491), (22699, 0.46069252490997314), (26989, 0.4500869810581207), (15580, 0.4489847421646118), (23529, 0.44809433817863464), (19009, 0.43329140543937683), (20077, 0.43276137113571167), (11193, 0.4265095889568329), (11077, 0.411053329706192)]


/var/folders/hd/dwzlt9p95njdvhxn511kq4fh0000gn/T/ipykernel_3643/2072519169.py:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  print(model.docvecs.most_similar([similar_words]))


In [71]:
similar_words

array([ 8.21499228e-02,  1.18160658e-01,  2.08531767e-01, -9.64381546e-02,
        1.27630711e-01,  1.07217506e-01, -3.86856534e-02, -7.15370327e-02,
        2.70377934e-01,  3.63785505e-01,  4.07267809e-01,  8.35922733e-02,
       -3.57877493e-01,  3.78631562e-01,  3.40689480e-01,  8.90214145e-02,
        1.21333841e-02, -3.11854750e-01,  5.73035218e-02, -2.57162124e-01,
       -8.04799050e-02,  3.94805044e-01,  2.09257215e-01, -1.01932414e-01,
        3.42676193e-01, -2.67439932e-01,  9.29514617e-02, -1.29939094e-01,
        3.87346983e-01, -2.15811372e-01, -7.25855231e-02,  2.68097281e-01,
       -5.11143684e-01, -5.27459860e-01, -7.59348227e-03, -3.98703754e-01,
       -2.95890123e-01,  1.21394783e-01,  4.36797231e-01,  2.27491140e-01,
        1.99736327e-01, -1.03583418e-01,  1.14006139e-01,  5.30392714e-02,
       -2.58662224e-01,  6.32780850e-01,  6.11870944e-01,  3.42602015e-01,
        5.21972403e-02, -3.95416230e-01, -1.55041412e-01,  4.78118323e-02,
        1.53026447e-01,  